In [ ]:
# 📌 GENDER MODEL TRAINING - CNN

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import numpy as np
from tqdm import tqdm

# Define CNN
class GenderCNN(nn.Module):
    def __init__(self):
        super(GenderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Dataset
class GenderVoiceDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        for label, gender in enumerate(["female", "male"]):
            gender_dir = os.path.join(root_dir, gender)
            for fname in os.listdir(gender_dir):
                if fname.endswith(".wav"):
                    self.samples.append((os.path.join(gender_dir, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        y, sr = librosa.load(file_path, sr=22050)
        y = librosa.util.fix_length(y, 22050 * 5)
        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        mel_db = librosa.power_to_db(mel, ref=np.max)
        mel_db = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min())
        mel_db = np.resize(mel_db, (128, 128))
        mel_tensor = torch.tensor(mel_db).unsqueeze(0).float()
        return mel_tensor, torch.tensor(label, dtype=torch.float32).unsqueeze(0)

# Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GenderCNN().to(device)
dataset = GenderVoiceDataset("data")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    correct, total, loss_total = 0, 0, 0
    for x, y in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        correct += ((pred > 0.5) == y).sum().item()
        total += y.size(0)
        loss_total += loss.item()
    print(f"Epoch {epoch+1} | Accuracy: {correct/total:.2%} | Loss: {loss_total:.4f}")

# Save model
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/gender_cnn.pth")
print("✅ Gender model saved.")
